In [48]:
import pydicom
import open3d as o3d
import glob
import numpy as np
import copy

In [49]:
paths = glob.glob("../data/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/2448190387/*.dcm")

In [54]:
pcds = []

for path in paths:
    slice = pydicom.read_file(path)
    img = np.expand_dims(slice.pixel_array, -1)
    pcd = o3d.geometry.PointCloud()
    x,y,z=np.where(img)
    
    index_voxel=np.vstack((x,y,z))
    grid_index_array=index_voxel.T
    pcd.points = o3d.utility.Vector3dVector(grid_index_array)
    colors = np.array([img[x,y,z] / 256 for x,y,z in grid_index_array]) 
    colors_rgb = np.stack([colors, colors, colors], axis=1)
    pcd.colors = o3d.utility.Vector3dVector(colors_rgb)
    
    d = slice.PixelSpacing

    X = np.array(list(slice.ImageOrientationPatient[:3]) + [0]) * d[0]
    Y = np.array(list(slice.ImageOrientationPatient[3:]) + [0]) * d[1]
    S = np.array(list(slice.ImagePositionPatient) + [1])
    
    transform_matrix = np.array([X, Y, np.zeros(len(X)), S]).T
    
    transformed = pcd.transform(transform_matrix)
    pcds.append(copy.deepcopy(transformed))

In [55]:
o3d.visualization.draw_geometries(pcds)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
